# Bayesian regression

Source: https://github.com/usptact/Infer.NET-BayesianRegression/blob/master/BayesianRegression/Program.cs  
Author: **Vladislavs Dovgalecs**   
A second example, below, is by **John Winn**.

### Links

http://infernet.azurewebsites.net/default.aspx    

## Load & open packages

https://fsprojects.github.io/Paket/

Packages in F# can be installed via `nuget.org` using the Paket dependency manager.
To install packages from NuGet you should first load the Packet package manager:

In [1]:
#load "Paket.fsx"

You can then have Paket install the packages:

In [2]:
Paket.Package
  [   // "Infer.NET" (pre Oct 2018 namespace)
      "Microsoft.ML.Probabilistic"
      "Microsoft.ML.Probabilistic.Compiler"
      //"Microsoft.ML.Probabilistic.FSharp"  // Not on nuget, yet?
      "NETStandard.Library" // Necessary??
  ]

In [3]:
#load "Paket.Generated.Refs.fsx"  // Do we need this?

### `FSharpWrapper`

See  https://github.com/dotnet/infer/blob/master/src/FSharpWrapper/FSharpWrapper.fs   
https://github.com/fsprojects/IfSharp/issues/146 (Thanks for help, Colin Gravill!)   
https://fsprojects.github.io/Paket/github-dependencies.html

In [4]:
Paket.GitHub ["dotnet/infer src/FSharpWrapper/FSharpWrapper.fs"]

In [5]:
// #load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs" // Not yet!?

### Assembly search paths and references

### Load & open packages

In [6]:
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic.Compiler/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/NETStandard.Library/build/netstandard2.0/ref"

In [7]:
#r "Microsoft.ML.Probabilistic"
#r "Microsoft.ML.Probabilistic.Compiler"
#r "netstandard"

## Model

Source: https://github.com/usptact/Infer.NET-BayesianRegression/blob/master/BayesianRegression/Program.cs  

The required classes are:
* `Microsoft.ML.Probabilistic`
 * `Distributions`
   * `Gaussian`
   * `VectorGaussian`
 * `Maths`
   * `Vector`
 * `Models`
   * `Variable`
   * `VariableArray`
   * `Range`

In [46]:
open System
open Microsoft.ML.Probabilistic
open Microsoft.ML.Probabilistic.Algorithms
open Microsoft.ML.Probabilistic.Distributions // VectorGaussian, Gamma
open Microsoft.ML.Probabilistic.Math          // Vector
open Microsoft.ML.Probabilistic.Models        // Variable, VariableArray, Range
open Microsoft.ML.Probabilistic.Factors
open Microsoft.ML.Probabilistic.Factors.Attributes

In [9]:
#load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs"

In [10]:
open Microsoft.ML.Probabilistic.FSharp

### Challenger O-ring data

http://archive.ics.uci.edu/ml/machine-learning-databases/space-shuttle/o-ring-erosion-only.data 

In [11]:
let temp = [| 66.; 70.; 69.; 68.; 67.; 72.; 73.; 70.; 57.; 63.; 70.; 78.; 67.; 53.; 67.; 75.; 70.; 81.; 76.; 79.; 75.; 76.; 58. |]
let distressInt = [| 0; 1; 0; 0; 0; 0; 0; 0; 1; 1; 1; 0; 0; 2; 0; 0; 0; 0; 0; 0; 0; 0; 1 |]  
let distress = distressInt |> Array.map (fun z -> float(z))

In [12]:
// put features into array of Vectors
let (xdata:Vector[]) = temp |> Array.map (fun z-> Vector.FromArray(z, 1.))         // including bias
//let xdata2 = temp |> Array.map (fun z-> Vector.FromArray([|z; 1.|]))    // Alternative array syntax

### Model variables

#### Prior distribution `w`

Define a prior distribution and attach that to `w` random variable

In [13]:
let wPrior = VectorGaussian(Vector.Zero(2), PositiveDefiniteMatrix.Identity(2))   
let w:Variable<Vector> = Variable.Random(wPrior)        

#### Noise

In [14]:
// hard-code variance
let noiseDist = Gamma(1., 2.)             
let noise:Variable<float> = Variable.Random(noiseDist)    

#### Features **`x`** and observations **`y`**

In [15]:
// set features "x" and observations "y" as observed in the model
let y:VariableArray<float> = Variable.Observed<float>(distress)  
let n = y.Range                           
let x:VariableArray<Vector> = Variable.Observed(xdata, n)       

Define `y` statistically: Gaussian RV array. Mean is defined by dot-product of param vector `w` and the feature vector `x[n]`

In [16]:
// define "y" statistically: Gaussian RV array. Mean is defined by dot-product of param vector "w" and the feature vector x[n]
let mean:Variable<float> = Variable.InnerProduct(w, x.[n])
let ydist:Variable<float> = Variable.GaussianFromMeanAndVariance(mean, noise)

In [17]:
y.[n] <- ydist   // y.set_Item(n, ydist)

### Training: parameter inference

https://dotnet.github.io/infer/apiguide/api/Microsoft.ML.Probabilistic.Models.InferenceEngine.html

In [18]:
let engine = InferenceEngine()
engine.Compiler.RecommendedQuality <- QualityBand.Experimental

Choose to dump the model as Scalable Vector Graphics (SVG) `.svg` or Model Specification Language (MSL)`.gv` files to the working directory
* [ShowFactorGraph](https://dotnet.github.io/infer/apiguide/api/Microsoft.ML.Probabilistic.Models.InferenceEngine.html#Microsoft_ML_Probabilistic_Models_InferenceEngine_ShowFactorGraph)
* [ShowMsl](https://dotnet.github.io/infer/apiguide/api/Microsoft.ML.Probabilistic.Models.InferenceEngine.html#Microsoft_ML_Probabilistic_Models_InferenceEngine_ShowMsl)

In [45]:
//engine.ShowFactorGraph <- true
//engine.ShowMsl <-true

In [42]:
// infer "w" posterior as a distribution
let wPosterior:VectorGaussian = engine.Infer<VectorGaussian>(w)
let noisePosterior:Gamma = engine.Infer<Gamma>(noise)

In [43]:
printfn "Distribution over w = \n %A" wPosterior

Distribution over w = 
 VectorGaussian(56.16 11.71, 0.1238   -0.01344)
                            -0.01344 0.01423 


In [44]:
printfn "Distribution over noise = \n %A"  noisePosterior

Distribution over noise = 
 Gamma(12.11, 0.01916)[mean=0.232]


### Prediction

In [22]:
// one data point
let tempTest = 31.

// RV for prediction
//let distressTest = Variable.Observed(tempTest)

In [23]:
// RV for feature vector
let xdataTest = Vector.FromArray(tempTest, 1.)   // MR.Infer.Maths.DenseVector
let (xTest:Variable<Vector>) = Variable.Observed<Vector>(xdataTest)

In [24]:
// set w distribution that was obtained from training
let wParam:Variable<Vector> = Variable.Random(wPosterior)          
let noiseParam:Variable<float> = Variable.Random(noisePosterior)

In [25]:
// RV for prediction
let mean2:Variable<float> = Variable.InnerProduct(wParam, xTest)

In [26]:
let distressTest:Variable<float> = Variable.GaussianFromMeanAndVariance(mean2, noiseParam)

In [27]:
// infer and print prediction distribution
engine.Infer(distressTest)

Gaussian(1.341, 0.4049)

Compiling model...done.


In [47]:
// Not sure why this fails? Perhaps the call to Infer does not simply return a string? 
// printfn "Test distress = \n %A" engine.Infer(distressTest)

### View model

#### `Models/Model_10_17_18_00_59_42_65.svg`

In [54]:
Util.Image("Models/Model_10_17_18_00_59_42_65.svg") |> Display  // Should work??

In [55]:
let html x = { Html = x } // IfSharp can render a record with single item of form {Html = "html-string"}

In [56]:
html """<img src="Models/Model_10_17_18_00_59_42_65.svg" alt="Model SVG">"""

In [50]:
// ![](Models/Model_10_17_18_00_59_42_65.svg "Model") (Markdown)

## Example 2

Author: John Winn

https://social.microsoft.com/Forums/en-US/3fed94a3-f0da-4dc7-993f-71d9b571d278/bayesian-linear-regression-migrated-from-communityresearchmicrosoftcom?forum=infer.net   
https://tminka.github.io/papers/minka-linear.pdf   

In [29]:
let (data:Vector[]) =  
    [|  [| 1.0; -3. |]; [| 1.0; -2.1 |]; [| 1.0; -1.3 |];  [| 1.0; 0.5 |];  
        [| 1.0; 1.2 |]; [| 1.0;  3.3 |]; [| 1.0;  4.4 |];  [| 1.0; 5.5 |] |]
    |> Array.map (fun z-> Vector.FromArray(z))

In [30]:
let rows:Range = Range(data.Length)

In [31]:
let x:VariableArray<Vector> = Variable.Constant(data, rows).Named("x")

In [32]:
let wMean = Vector.FromArray([| 0.; 0.|])
let wPrecision = PositiveDefiniteMatrix.Identity(2)
let w:Variable<Vector> = Variable.VectorGaussianFromMeanAndPrecision(wMean,wPrecision).Named("w");

In [33]:
let y:VariableArray<float> = Variable.Array<float>(rows)
y.[rows] <- Variable.GaussianFromMeanAndVariance(Variable.InnerProduct(x.[rows], w),1.0)
y.ObservedValue <- [| 30.; 45.; 40.; 80.; 70.; 100.; 130.; 110. |] 

In [48]:
let engine2 = InferenceEngine(VariationalMessagePassing())
let postW = engine2.Infer<VectorGaussian>(w)
printfn "Posterior over the weights: \n %A" postW

Compiling model...done.
Posterior over the weights: 
 VectorGaussian(56.16 11.71, 0.1238   -0.01344)
                            -0.01344 0.01423 
